In [1423]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [1424]:
term_deposit = pd.read_csv('bank-additional-full.csv',sep= ";")
term_deposit = term_deposit.drop(['emp.var.rate','nr.employed','cons.conf.idx'],axis  =1)
term_deposit

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,cons.price.idx,euribor3m,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,93.994,4.857,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,93.994,4.857,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,93.994,4.857,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,93.994,4.857,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,93.994,4.857,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41183,73,retired,married,professional.course,no,yes,no,cellular,nov,fri,334,1,999,0,nonexistent,94.767,1.028,yes
41184,46,blue-collar,married,professional.course,no,no,no,cellular,nov,fri,383,1,999,0,nonexistent,94.767,1.028,no
41185,56,retired,married,university.degree,no,yes,no,cellular,nov,fri,189,2,999,0,nonexistent,94.767,1.028,no
41186,44,technician,married,professional.course,no,no,no,cellular,nov,fri,442,1,999,0,nonexistent,94.767,1.028,yes


In [1425]:
months = 'mar,apr,may,jun,jul,aug,sep,oct,nov,dec'.split(',')
month_num = [3,4,5,6,7,8,9,10,11,12]
d = dict(zip(months, month_num))
term_deposit['month_num'] = [d[each] for each in term_deposit['month'] if each in d]

In [1426]:
term_deposit = term_deposit.sort_values('month_num')

In [1427]:
term_deposit

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,cons.price.idx,euribor3m,y,month_num
27786,39,admin.,single,high.school,no,no,no,cellular,mar,tue,119,5,999,0,nonexistent,92.843,1.687,no,3
27703,38,admin.,single,university.degree,no,yes,no,cellular,mar,mon,69,1,999,0,nonexistent,92.843,1.811,no,3
27702,31,admin.,single,university.degree,no,yes,no,cellular,mar,mon,145,1,999,0,nonexistent,92.843,1.811,no,3
27701,28,technician,single,university.degree,no,no,no,cellular,mar,mon,142,2,999,0,nonexistent,92.843,1.811,no,3
27700,26,admin.,married,university.degree,no,yes,no,cellular,mar,mon,154,1,999,0,nonexistent,92.843,1.811,yes,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39066,52,technician,married,high.school,no,no,no,cellular,dec,fri,406,1,999,1,failure,92.713,0.710,yes,12
39065,21,student,single,university.degree,no,yes,no,cellular,dec,fri,379,3,6,1,success,92.713,0.710,yes,12
39064,28,admin.,single,university.degree,no,yes,yes,cellular,dec,fri,454,6,999,1,failure,92.713,0.710,no,12
39062,76,retired,married,university.degree,no,unknown,unknown,cellular,dec,thu,184,6,999,1,failure,92.713,0.712,no,12


In [1428]:
term_deposit = term_deposit.drop('contact',axis = 1)
train = term_deposit[:-10000]
valid = term_deposit[-10000:-5000]
test = term_deposit[-5000:]

In [1429]:
y_train = train['y']
X_train = train.drop('y',axis = 1)

y_val = valid['y']
X_val = valid.drop('y',axis = 1)

y_test = test['y']
X_test = test.drop('y',axis = 1)

In [1430]:
X_train_cat_df = X_train.select_dtypes(include = 'object')
X_train_cat_df = pd.get_dummies(X_train_cat_df, drop_first= True)

X_val_cat_df = X_val.select_dtypes(include = 'object')
X_val_cat_df = pd.get_dummies(X_val_cat_df, drop_first= True)


X_test_cat_df = X_test.select_dtypes(include = 'object')
X_test_cat_df = pd.get_dummies(X_test_cat_df, drop_first= True)

In [1431]:
X_train = pd.concat([X_train.select_dtypes(include = 'number'), X_train_cat_df], axis = 1)

X_val = pd.concat([X_val.select_dtypes(include = 'number'), X_val_cat_df], axis = 1)

X_test = pd.concat([X_test.select_dtypes(include = 'number'), X_test_cat_df], axis = 1)

In [1432]:
missing_cols = set(X_val.columns) - set(X_train.columns)
for col in missing_cols:
    X_train[col] = 0
X_train = X_train[X_val.columns]    

In [1433]:
missing_col = set(X_train.columns) - set(X_test.columns)
for col in missing_col:
    X_test[col] = 0  
X_test = X_test[X_train.columns]

In [1434]:
#missing_co = set(X_test.columns) - set(X_train.columns)
#for col in missing_co:
 #   X_train[col] = 0

In [1435]:
#missing_c = set(X_train.columns) - set(X_val.columns)
#for col in missing_c:
#    X_val[col] = 0

In [1436]:
from sklearn.preprocessing import StandardScaler

In [1437]:
scaler = StandardScaler()
scaled_data_train = scaler.fit_transform(X_train)
X_train = pd.DataFrame(scaled_data_train, columns = X_train.columns)

scaled_data_val = scaler.fit_transform(X_val)
X_val = pd.DataFrame(scaled_data_val, columns = X_val.columns)

scaled_data_test = scaler.fit_transform(X_test)
X_test = pd.DataFrame(scaled_data_test, columns = X_test.columns)

In [1438]:
y_train

27786     no
27703     no
27702     no
27701     no
27700    yes
        ... 
22856     no
22857     no
22858     no
22859    yes
22860     no
Name: y, Length: 31188, dtype: object

In [1439]:
from imblearn.over_sampling import SMOTE

In [1440]:
sm = SMOTE(random_state= 27, ratio= 1.0)

In [1441]:
X_train, y_train = sm.fit_sample(X_train, y_train)

In [1442]:
#X_train = pd.DataFrame(X_train,columns = X_val.columns)
X_train = pd.DataFrame(X_train,columns = X_val.columns)

In [1443]:
from collections import Counter
Counter(y_train)

Counter({'no': 27867, 'yes': 27867})

In [1444]:
#from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [1445]:
#lr = LogisticRegression(solver = 'liblinear')
xgb = XGBClassifier(n_estimators= 180, learning_rate= 0.01, gamma= 0.5, max_depth=3)

In [1446]:
xgb.fit(X_train, y_train, eval_set=[(X_val,y_val)], early_stopping_rounds= 5)
#lr.fit(X_train,y_train)

[0]	validation_0-error:0.3078
Will train until validation_0-error hasn't improved in 5 rounds.
[1]	validation_0-error:0.3078
[2]	validation_0-error:0.3078
[3]	validation_0-error:0.3078
[4]	validation_0-error:0.3078
[5]	validation_0-error:0.3078
Stopping. Best iteration:
[0]	validation_0-error:0.3078



XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0.5,
              learning_rate=0.01, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=180, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [1447]:
X_test = X_test[X_train.columns]

In [1457]:
pred = xgb.predict(X_test)

In [1449]:
from sklearn.metrics import classification_report, confusion_matrix

In [1458]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

          no       0.97      0.61      0.75      4180
         yes       0.31      0.89      0.46       820

    accuracy                           0.66      5000
   macro avg       0.64      0.75      0.60      5000
weighted avg       0.86      0.66      0.70      5000



In [1459]:
print(confusion_matrix(y_test,pred))

[[2546 1634]
 [  89  731]]


In [1468]:
from sklearn.metrics import recall_score

In [1461]:
from sklearn.preprocessing import LabelBinarizer

In [1462]:
lb = LabelBinarizer()
y_v = np.array([number[0] for number in lb.fit_transform(y_test)])

pr = np.array([number[0] for number in lb.fit_transform(pred)])

In [1463]:
recall_score(y_v,pr)

0.8914634146341464